## Initialization code

In [1]:
%matplotlib inline

In [2]:
import os
import tempfile
import pandas as pd
import numpy as np
import networkx as nx

## Compilation and test of waf command

The file iterative_avg_consensus.cc must be copied in the scratch folder of ns3

In [3]:
!../cmake-build-debug/iterative_avg_consensus --PrintHelp

iterative_avg_consensus [Program Arguments] [General Arguments]

Program Arguments:
    --nNodes:            Number of nodes in the simulation [2]
    --secsToRun:         Number of seconds to simulate [10]
    --makhoul:           Use the example of makhoul thesis [0]
    --path:              Output path [/tmp]
    --self_stab:         Self-stabilization mode [0]
    --positionRng:       Define the random generator for the position allocator [ns3::UniformRandomVariable[Min=0.0|Max=1000.0]]
    --distance:          Define the maximum signal range [250]
    --async:             Set async mode for diffusion algorithm [0]
    --seed:              Set seed value (simulation reproducibility) [42]
    --run_id:            Set run id (simulation reproducibility) [0]
    --graph_correction:  Run the graph correction algorithm for non-connected components [1]
    --nbranch:           Generates random network without distances (with nbranch > 0) [0]
    --node_density:      Node density / km2 [1

## Simulation meta command

NS_LOG=IterativeAvgConsensus=level_info 

def start_simulation(nnodes=20, distance=30, secs=50, test=0, self_stab=0, async=0, run_id=0,
                    graph_correction=1):
    if test > 0:
        nnodes = 4
    path = tempfile.mkdtemp()
    cmd = '''../cmake-build-debug/iterative_avg_consensus \
--nNodes={nnodes} --distance={distance} \
--secsToRun={secs} --makhoul={test} --path={path} --self_stab={self_stab} --async={async} \
--run_id={run_id} --graph_correction={graph_correction}  2>&1 >/dev/null \
| grep "^TR" > {path}/ts.tsv'''.format(nnodes=nnodes,
                                                                            distance=distance,
                                                                           secs=secs,
                                                                           path=path,
                                                                           test=test,
                                                                           self_stab=self_stab,
                                                                           async=async,
                                                                           run_id=run_id,
                                                                           graph_correction=graph_correction)
    print cmd
    os.system(cmd)
    trace_df = pd.read_csv("{path}/ts.tsv".format(path=path), sep='\t', 
                       names=["tag", "node", "sim_time", "nit", "nmsg", "z", "x", "energy", "nmove", "itnmove"])

    G = nx.Graph()    
    positions = {}
    
    with open("{path}/graph.tsv".format(path=path), "r") as fp:
        # Get positions
        for i in range(nnodes):
            nid, x, y = fp.readline().replace("\n", "").split("\t")
            G.add_node("node%s" %nid, pos=(x, y))
            positions["node%s" %nid] = (float(x), float(y))
        # Get connections
        for connection in fp:
            fn, tn = connection.replace("\n", "").split("\t")
            G.add_edge("node%s" %fn, "node%s" %tn)
        G.positions = positions
    return trace_df, G

def get_epsilon(df, col, sim_avg=1.0):
    df["eps"] = abs(df.x - sim_avg)
    return df[[col, "eps"]].groupby(col).agg(np.max)

def get_epsilon_min(df, col, thr=10e-4, sim_avg=1.0):
    pos, _ = np.where(get_epsilon(df, col, sim_avg)<thr)
    if len(pos) > 0:
        return pos[0]
    else:
        return None

## Plotting commands

def compute_convergence(df, epsilon=0.1, sim_avg=1.0, plot=False):
    # 50 ms bucket    
    # Conv vs time
    conv_df = df.copy()
    conv_df["sim_step"] = (conv_df.sim_time/50).astype(int)
    outdf = conv_df[["sim_step", "node", "x"]].groupby(["sim_step", "node"]).agg(np.mean).reset_index()

    #    eps_min = get_epsilon_min(outdf, col="sim_step", thr=epsilon, sim_avg=sim_avg)
    vmin = outdf[["x", "sim_step"]].groupby("sim_step").agg(np.max) - outdf[["x", "sim_step"]].groupby("sim_step").agg(np.min)
    
    try:
        eps_min = np.where(vmin.x < epsilon)[0][0]
    except:
        eps_min = None

#    if v is not None:
#        outdf = outdf[outdf.sim_step<=v]
    outdf.sim_step *= 50
    conv_df.sim_step *= 50
    pv_df = outdf.pivot(index="sim_step", columns="node", values="x")
    pv_df["nmsg"] = conv_df[["nmsg", "sim_step"]].groupby("sim_step").agg(np.sum)
    
    if plot:
        pv_df.plot(secondary_y="nmsg", title="Convergence vs time", grid=True)
        
    pv_df["nmove"] = conv_df[["nmove", "sim_step"]].groupby("sim_step").agg(np.sum)
    pv_df["itnmove"] = conv_df[["itnmove", "sim_step"]].groupby("sim_step").agg(np.max)
    
    metrics_df = pv_df[["nmsg", "nmove", "itnmove"]].copy()
    
    pv_df.drop(["nmsg"], axis=1, inplace=True)
    pv_df["energy"] = conv_df[["energy", "sim_step"]].groupby("sim_step").agg(np.mean)
    if plot:    
        pv_df.plot(secondary_y="energy", title="Convergence vs time / energy", grid=True)
    metrics_df["energy"] = pv_df["energy"]
    
    delta_df = outdf.copy()
#    delta_df["delta"] = abs(delta_df.x - sim_avg)
    delta_plt = delta_df[["x", "sim_step"]].groupby("sim_step").agg(np.max) - delta_df[["x", "sim_step"]].groupby("sim_step").agg(np.min)
#    delta_plt = delta_df[["delta", "sim_step"]].groupby("sim_step").agg(np.max)
    if plot:    
        delta_plt.plot(title="Maximum delta vs time", grid=True)
#    metrics_df["delta"] = delta_plt["delta"]
    metrics_df["delta"] = delta_plt["x"]
    
    x_mean = delta_df[["x", "sim_step"]].groupby("sim_step").agg(np.mean).reset_index()
    x_mean.rename(columns={"x": "x_mean"}, inplace=True)
    outside_df = delta_df[["x", "sim_step"]].merge(x_mean, how="left", on="sim_step")
    outside_df["x_out"] = (abs(outside_df["x"] - outside_df["x_mean"]) > 0.1).astype(int)
    
    metrics_df["nb_x_out"] = outside_df[["sim_step", "x_out"]].groupby("sim_step").agg(np.sum)
    
    return metrics_df, eps_min

    
    '''
    outdf = df.pivot(index="nit", columns="node", values="x")
    outdf["nmsg"] = df[["nmsg", "nit"]].groupby("nit").agg(np.sum)
    if v is not None:
        outdf = outdf[outdf.index<=v]
    outdf.plot(secondary_y="nmsg", title="Convergence vs iterations", grid=True)
    '''
    
    flt_out = outdf.drop(["nmsg"], axis=1)
    flt_out["min_max_delta"] = df[["x", "sim_step"]].groupby("sim_step").agg(np.max) - df[["x", "sim_step"]].groupby("sim_step").agg(np.min)
    pd.DataFrame(index=outdf.nmsg, data=flt_out.values, 
                 columns=flt_out.columns).plot(secondary_y="min_max_delta", 
                                               title="Convergence vs # messages exchanged",
                                              legend='reverse', grid=True)

def plot_graph(G):
    nx.draw(G, G.positions, with_labels = True)

def plot_minmax(df):
    outdf = df.pivot(index="nit", columns="node", values="x")
    outdf["delta"] = df[["x", "nit"]].groupby("nit").agg(np.max) - df[["x", "nit"]].groupby("nit").agg(np.min)
    outdf["delta"].plot()

# Experiments

In [31]:
import os

In [32]:
def experiment(meta_args, epsilon=10e-3, sim_avg=1, nbruns=1, force_run_id=None, xp_name="data"):
    try:
        os.mkdir(xp_name)
    except:
        pass
    if force_run_id is not None:
        nbruns = 1
    for run_id in range(nbruns):
        if force_run_id is not None:
            run_id = force_run_id
        # diffusion
        trace_results, G = start_simulation(run_id=run_id, **meta_args)
        degree_diff = G.degree(G.nodes())
        diam = nx.diameter(G)
        epsilon = 0.1 * diam
        print "Epsilon", epsilon
        metrics_diffusion, eps_min_diff = compute_convergence(trace_results, epsilon=epsilon, sim_avg=sim_avg)
        eps_min_diff = metrics_diffusion.shape[0]
        # self-tab
        trace_results, G = start_simulation(run_id=run_id, self_stab=1, **meta_args)
        degree_sstab = G.degree(G.nodes())        
        metrics_sstab, eps_min_sstab = compute_convergence(trace_results, epsilon=epsilon, sim_avg=sim_avg)
        eps_min_sstab = metrics_sstab.shape[0]

        mg = pd.merge(metrics_diffusion, metrics_sstab, how='outer', 
                      left_index=True, right_index=True, suffixes=('_diff', '_sstab'))
        mg["run_id"] = run_id
        mg["xp_name"] = xp_name
        mg["eps_min_diff"] = eps_min_diff
        mg["eps_min_sstab"] = eps_min_sstab       
        mg["degree_diff"] = np.mean(degree_diff.values())
        mg["degree_sstab"] = np.mean(degree_sstab.values())
        mg["diameter"] = diam
        mg["epsilon"] = epsilon  
        mg.fillna(method='pad', inplace=True)
        mg.to_csv("%s/run_%d.csv" %(xp_name, run_id), sep='\t')
    return mg

## Exper

In [33]:
%pdb

Automatic pdb calling has been turned OFF


In [51]:
for nnode in [200]:#[50, 100, 200, 400, 600, 800]:
    meta_args = dict(nnodes=nnode, distance=300, secs=100, graph_correction=1)
    mg = experiment(meta_args, nbruns=2, xp_name="nnodes_%d-distance_42" %nnode)

../cmake-build-debug/iterative_avg_consensus --nNodes=200 --distance=300 --secsToRun=100 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_5LnsC --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_5LnsC/ts.tsv
Epsilon 6.3
../cmake-build-debug/iterative_avg_consensus --nNodes=200 --distance=300 --secsToRun=100 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpIDbN8r --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpIDbN8r/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=200 --distance=300 --secsToRun=100 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpAYk1z6 --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpAYk1z6/ts.tsv
Epsilon 5.5
../cmake-buil

In [52]:
mg

node,nmsg_diff,nmove_diff,itnmove_diff,energy_diff,delta_diff,nb_x_out_diff,nmsg_sstab,nmove_sstab,itnmove_sstab,energy_sstab,delta_sstab,nb_x_out_sstab,run_id,xp_name,eps_min_diff,eps_min_sstab,degree_diff,degree_sstab,diameter,epsilon
sim_step,,,,,,,,,,,,,,,,,,,,
50,400,135,134,1.000,9.825,200,400.0,0.0,0.0,1.000000,9.825,200.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
100,1200,263,128,0.999,7.870,189,992.0,52.0,51.0,0.999000,9.825,194.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
150,2000,412,149,0.999,7.140,193,1497.0,104.0,52.0,0.998955,9.147,191.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
200,2800,556,144,0.998,6.670,192,2068.0,162.0,58.0,0.998145,9.147,182.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
250,3600,700,144,0.998,6.340,189,2655.0,222.0,60.0,0.998015,8.017,180.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
300,4400,850,150,0.998,6.030,189,3233.0,281.0,59.0,0.997275,7.566,189.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
350,5200,991,141,0.997,5.760,185,3789.0,337.0,56.0,0.997140,5.440,183.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
400,6000,1134,143,0.997,5.530,177,4344.0,393.0,56.0,0.996480,5.300,177.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5
450,6800,1278,144,0.996,5.370,177,4883.0,448.0,55.0,0.996280,5.000,175.0,1,nnodes_200-distance_42,247,138,2.0,2.0,55,5.5


In [54]:
## Generating experiments that can be started with parallel function

EPSILON=10e-3
SIM_AVG=1.

line_no = 1
xpid = 19
distance = 300

with open("scaling_experiments.sh", "w+") as fp:
    for run_id in range(50):
        for nnode in [100]:#50, 100, 200, 300, 400, 600]:
            fp.write("python run_experiment.py ../cmake-build-debug/iterative_avg_consensus %d %d %d %d\n" %(run_id, nnode, xpid, distance))
            line_no += 1